##ДЗ_10 “Перевод без механизма внимания - SEQ2SEQ модель”

**Разобраться с моделью перевода (без механизма внимания) как она устроена, запустить для перевода с русского на английский (при желании можно взять другие пары языков).**

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

### Загрузим датасет.



In [ ]:
!wget http://www.manythings.org/anki/rus-eng.zip

--2022-11-30 14:46:49--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15011848 (14M) [application/zip]
Saving to: ‘rus-eng.zip.1’

rus-eng.zip.1       100%[===================>]  14.32M  8.26MB/s    in 1.7s    

2022-11-30 14:46:52 (8.26 MB/s) - ‘rus-eng.zip.1’ saved [15011848/15011848]



In [ ]:
!mkdir rus-eng
!unzip rus-eng.zip -d rus-eng/

mkdir: cannot create directory ‘rus-eng’: File exists
Archive:  rus-eng.zip
replace rus-eng/rus.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: rus-eng/rus.txt         
replace rus-eng/_about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: rus-eng/_about.txt      


In [ ]:
!ls /content/rus-eng/ -lah

total 72M
drwxr-xr-x 2 root root 4.0K Nov 30 14:47 .
drwxr-xr-x 1 root root 4.0K Nov 30 14:46 ..
-rw-r--r-- 1 root root 1.5K Sep  6 03:10 _about.txt
-rw-r--r-- 1 root root  72M Sep  6 03:10 rus.txt


In [ ]:
path_to_file = "/content/rus-eng/rus.txt"

In [ ]:
#функция препроцессинга
def preprocess_sentence(w):
  #переводим предложение к нижнему регистру и удалем начальные и конечные пробелы
  w = w.lower().strip()

  # отделяем пробелом слово и следующую за ним пунктуацию
  # пример: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # все, кроме букв и знаков пунктуации, заменяем пробелом
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  #удаляем лишние пробелы в начале и конце
  w = w.strip()

  # создаем начало и конец последовательности
  # теперь модель знает, где начинать и заканчивать предсказания
  w = '<start> ' + w + ' <end>'
  return w

In [ ]:
preprocess_sentence("I can't go.")

"<start> i can't go . <end>"

In [ ]:
# 1. Убираем акценты
# 2. Очищаем предложения
# 3. Возвращаем пары слов в формате [ENG, RUS]
def create_dataset(path, num_examples):
  #считываем строки файла
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  #каждую строку разделяем на пробелы, берем первые 2 слова, препроцессим их и возвращаем пару
  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [ ]:
en, ru = create_dataset(path_to_file, None)
print(en[0])
print(ru[0])

<start> go . <end>
<start> марш ! <end>


In [ ]:
def tokenize(lang):
  #токенизируем текст, отфильтровываем пробелы
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  #обновляем внутренний словарь на основе lang
  lang_tokenizer.fit_on_texts(lang)

  #преобразуем каждый элемент из lang в последовательность чисел
  tensor = lang_tokenizer.texts_to_sequences(lang)

  #преобразуем тензор в матрицу (кол-во тензоров * max-длина), 
  #при этом короткие последовательности заполняем нулями сзади, а длинные -- обрезаем сзади
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [ ]:
def load_dataset(path, num_examples=None):
  # создаем очищенные анг (выходные), русские (входные) пары
  targ_lang, inp_lang = create_dataset(path, num_examples)
  #применяем токенизацию к каждому элементы из пары
  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

**Для ускорения обучения установим лимит на размер датасета.**

In [ ]:
len(en), len(ru)

(451436, 451436)

In [ ]:
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Максимальная длина тензоров
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [ ]:
# Создаем train и validation датасеты
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# размеры датасетов
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


In [ ]:
# выведем токен и его индекс
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [ ]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[10])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[10])

Input Language; index to word mapping
1 ----> <start>
96 ----> давайте
19419 ----> закругляться
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
66 ----> let's
2394 ----> wrap
660 ----> things
54 ----> up
3 ----> .
2 ----> <end>


In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

#из каждого элемента (input_tensor_train, target_tensor_train) создаем тензор
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
#разбиваем датасет на батчи (по 64), удаляя последний неполный батч
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 15]), TensorShape([64, 11]))

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=False,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return state

  def initialize_hidden_state(self):
    #создаем тензор из нулей размера (батч, кол-во ячеек)
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# инициализируем начальное скрытое состояние (из нулей)
sample_hidden = encoder.initialize_hidden_state()
# применяем энкодер к входному батчу и скрытому состоянию
sample_hidden = encoder(example_input_batch, sample_hidden)
# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder Hidden state shape: (batch size, units) (64, 1024)


In [ ]:
# декодер без внимания
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

  def call(self, x, hidden):
    # x shape после прохождения через эмбеддинг == (batch_size, 1, embedding_dim)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # отправляем в GRU входные данные и скрытое состояние (от энкодера)
    # выход GRU (batch_size, timesteps, units)
    # размер возвращаемого внутреннего состояния (batch_size, units)
    output, state = self.gru(x, initial_state=hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

#применяем декодер к случайному батчу из равномерного распределения (батч,1) и выходу энкодера
decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)



In [ ]:
decoder_sample_x.shape

TensorShape([64, 7335])

In [ ]:
decoder_sample_h.shape

TensorShape([64, 1024])

**Определим оптимизатор и функцию потерь.**

In [ ]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  #делаем инверсию значений сравнения каждого из real с нулем (возвращается true или false)
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  #применяем функцию ошибок к реальным данным и предсказанным
  loss_ = loss_object(real, pred)

  #приводим тензор mask к новому типу loss_.dtype
  mask = tf.cast(mask, dtype=loss_.dtype)
  #умножаем loss_ на mask
  loss_ *= mask
  
  # возвращаем среднее значениe всех элементов
  return tf.reduce_mean(loss_)

**Создадим директорию для сохранения checkpoint.**

In [ ]:
checkpoint_dir = './training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0
  #перечисляем операции для автоматического дифференцирования
  with tf.GradientTape() as tape:
    #перечисляем операции для автоматического дифференцирования
    enc_hidden = encoder(inp, enc_hidden)
    #помещаем его в скрытое состояние decoder
    dec_hidden = enc_hidden
    #формируем вход декодера:
    #берем список длины батч из индексов тега  (1)
    #приписываем списку размерность 1
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    #Teacher forcing - выводим target в качестве следующего входа
    for t in range(1, targ.shape[1]):
      #Teacher forcing - выводим target в качестве следующего входа
      predictions, dec_hidden = decoder(dec_input, dec_hidden)
      #считаем функцию потерь 
      loss += loss_function(targ[:, t], predictions)

      #используем teacher forcing (приписываем списку размерность 1)
      #посылаем dec_input на вход декордера 
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables
  #вычисляем градиенты loss по variables
  gradients = tape.gradient(loss, variables)
  #оптимизатор применяет подсчитанные градиенты
  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 50

for epoch in range(EPOCHS):
  start = time.time()

  #инициализируем входное скрытое состояние (из нулей) размера (батч, кол-во рекуррентных ячеек)
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    #делаем шаг обучения. находим ошибку за эпоху
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  #сохраняем чекпоинт каждые 25 эпох
  if (epoch + 1) % 25 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.6656
Epoch 1 Batch 100 Loss 1.9159
Epoch 1 Batch 200 Loss 1.8898
Epoch 1 Batch 300 Loss 1.6953
Epoch 1 Batch 400 Loss 1.6190
Epoch 1 Batch 500 Loss 1.5065
Epoch 1 Batch 600 Loss 1.4696
Epoch 1 Batch 700 Loss 1.3053
Epoch 1 Batch 800 Loss 1.3094
Epoch 1 Batch 900 Loss 1.2544
Epoch 1 Batch 1000 Loss 1.2061
Epoch 1 Batch 1100 Loss 1.2703
Epoch 1 Batch 1200 Loss 1.1564
Epoch 1 Loss 1.4936
Time taken for 1 epoch 68.1933331489563 sec

Epoch 2 Batch 0 Loss 0.9884
Epoch 2 Batch 100 Loss 0.9358
Epoch 2 Batch 200 Loss 1.0190
Epoch 2 Batch 300 Loss 0.8937
Epoch 2 Batch 400 Loss 0.8032
Epoch 2 Batch 500 Loss 0.8495
Epoch 2 Batch 600 Loss 0.8659
Epoch 2 Batch 700 Loss 0.8570
Epoch 2 Batch 800 Loss 0.8087
Epoch 2 Batch 900 Loss 0.6899
Epoch 2 Batch 1000 Loss 0.6543
Epoch 2 Batch 1100 Loss 0.7605
Epoch 2 Batch 1200 Loss 0.5786
Epoch 2 Loss 0.8125
Time taken for 1 epoch 57.405874490737915 sec

Epoch 3 Batch 0 Loss 0.4910
Epoch 3 Batch 100 Loss 0.4766
Epoch 3 Batch 200 Loss 0.486

In [ ]:
def evaluate(sentence):
  # выполняем препоцессинг предложения
  attention_plot = np.zeros((max_length_targ, max_length_inp))
  #разбиваем предложение по пробелам и составляем список индексов каждого слова
  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  #заполняем inputs нулями справа до максимальной длины входного текста
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  #преобразуем inputs в тензор
  inputs = tf.convert_to_tensor(inputs)

  result = ''
  #инициализируем входной hidden из нулей размера (1, units)
  hidden = [tf.zeros((1, units))]
  #подаем inputs и hidden в encoder
  enc_hidden = encoder(inputs, hidden)

  #инициализируем входной hidden декодера -- выходной hidden энкодера
  dec_hidden = enc_hidden
  #вход декодера -- список [индекс start] размера(1,1)
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    #получаем выход декодера
    predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # storing the attention weights to plot later on
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += targ_lang.index_word[predicted_id] + ' '

    #заканчиваем на токене end
    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence

    #предсказанный predicted ID подаем обратно в декодер (размер (1,1))
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

**Напишем функцию для перевода текста.**

In [ ]:
def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

**Загрузим последний сохраненный чекпойнт .**

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

### Протестируем работу модели.

In [ ]:
translate('Хорошая погода.')

Input: <start> здесь хорошо . <end>
Predicted translation: it's good here . <end> 


In [ ]:
translate('Я не смогу поехать.')

Input: <start> я не смогу поехать . <end>
Predicted translation: i can't go . <end> 


In [ ]:
translate(u'Вы еще дома?')

Input: <start> вы еще дома ? <end>
Predicted translation: are you still home ? <end> 


In [ ]:
translate(u'Вы все еще дома?')

Input: <start> вы все еще дома ? <end>
Predicted translation: are you still home ? <end> 


In [ ]:
translate(u'Вы уже дома?')

Input: <start> вы уже дома ? <end>
Predicted translation: are you at home yet ? <end> 


In [ ]:
translate(u'Попробуй сделать это.')

Input: <start> попробуй сделать это . <end>
Predicted translation: try to do that . <end> 


In [ ]:
translate(u'Я люблю, когда идет снег.')

Input: <start> я люблю , когда идет снег . <end>
Predicted translation: i like snow a lot . <end> 


In [ ]:
translate(u'Это хороший фильм.')

Input: <start> это хороший фильм . <end>
Predicted translation: this is a good movie . <end> 


In [ ]:
translate(u'Это не совсем хороший фильм.')

Input: <start> это не совсем хороший фильм . <end>
Predicted translation: it's not that movie . <end> 


In [ ]:
translate(u'Это совсем нехороший фильм.')

Input: <start> это совсем нехороший фильм . <end>
Predicted translation: it's not really fussy . <end> 


In [ ]:
translate(u'Любите Родину, вашу мать!')

Input: <start> любите родину , вашу мать ! <end>
Predicted translation: i wish you love . <end> 


In [ ]:
translate(u'На улице темнота, ни зги не видно.')

KeyError: ignored

## ВЫВОД: Модель вполне работоспособная и неплохо справляется с простыми короткими фразами. Если попадается слово, отсутствующее в словаре модели, выпадает ошибка.